In [1]:
#Imports
import os
import pandas as pd 
import matplotlib.pyplot as plt 
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import Dataset
from PIL import Image


from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd


In [2]:
# reading csv file 
labels = pd.read_csv('../input/clothing-dataset-full/images.csv')
labels.head()

In [4]:
# Adding extention 
labels['image'] = labels['image'] + '.jpg'
# Creating dataframe with image_path, and lable_class
label_df = labels[['image', 'label']]

In [4]:
label_df.head()

In [5]:
# Finding list of classes
classes=list(label_df['label'].unique())
classes

In [5]:
# Classes and their counts within the dataset
label_df['label'].value_counts()

In [6]:
#Mapping class name to class number
repl={}
for i in range(len(classes)):
    repl[classes[i]]=i
    
repl

In [7]:
label_df.head()

In [ ]:
# Assign label number 
for i in range(len(label_df)):
    label_df['label'][i]=repl[label_df['label'][i]]

In [9]:
label_df.head()

In [10]:
# splite data
from sklearn.model_selection import train_test_split
train_label_df, test_label_df = train_test_split(label_df, test_size=0.10)

In [11]:
print(len(train_label_df))
print(len(test_label_df))

In [12]:
print(type(train_label_df))
print(train_label_df.head())

In [13]:
# create train and test files
train_label_df.to_csv ('./train_csv3.csv', index = False, header=True)
test_label_df.to_csv ('./test_csv3.csv', index = False, header=True)

In [15]:
PATH = '../input/clothing-dataset-full/images_original'
PATH = os.path.join(PATH, '9e530520-3973-44d0-b630-e213c809b854.jpg')
img  = Image.open(PATH)
img

In [16]:
# creating Dataset class 
class DressDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")
        y_label = torch.tensor(self.annotations.iloc[index, 1])
        #float()

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

In [17]:
transform = transforms.Compose(
    [transforms.Resize((32, 32)),
     transforms.ToTensor(),# Tensor, values between 0 and 1
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])# mean and std of ***ImageNet*** data set

# assign batch size
batch_size = 16

In [18]:
# create training data frame
train_df = pd.read_csv('./train_csv3.csv')

In [19]:
train_df.head()

In [20]:
trainset = DressDataset(
    root_dir='../input/clothing-dataset-full/images_original',
    annotation_file='./train_csv3.csv',
    transform = transform
    )

In [23]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [25]:
testset = DressDataset(root_dir='../input/clothing-dataset-full/images_original/', annotation_file='./test_csv3.csv', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [26]:
data_loaders = {'train':trainloader, 'test':testloader} 

In [40]:
model = torchvision.models.vgg16(pretrained=True)

In [41]:
model

In [93]:
import torch.nn as nn
from torch.nn import Flatten, Linear, ReLU, Conv2d,MaxPool2d, AdaptiveAvgPool2d
# The new Added convolution layer
Added_Layers = torch.nn.modules.Sequential(Conv2d(in_channels=512,
                                                 out_channels=512,
                                                    kernel_size=3,
                                                        padding=1,
                                                         stride=1),
                                    ReLU(),
                                    AdaptiveAvgPool2d(output_size=(7, 7))
                                    )
# Freeze all layers
for param in model.parameters():
    param.requires_grad = False




# Parameters of newly constructed modules have requires_grad=True by default
# Replace the last layers
model.avgpool =Added_Layers
model.classifier = nn.Linear(25088, 20)  
# Optimize only the parameters of ''classifier', and 'avgpool'
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#model.classifier.parameters()

In [63]:
# check state of gradient of parametrs
for name, param in model.named_parameters():
    print('Name: ', name,  'Requires_Grad: ', param.requires_grad)

In [64]:
# unFreeze all layers
for param in model.parameters():
    param.requires_grad = True

In [65]:
for name, param in model.named_parameters():
    print('Name: ', name,  'Requires_Grad: ', param.requires_grad)

In [58]:
model

In [44]:
model.train()
model = model.cuda()

In [71]:
def optimization(num_of_epochs):
    
    train_step_losses  = []
    train_epoch_losses = []

    test_step_losses   = []
    test_epoch_losses  = []

    print_every = 10
    num_epochs = num_of_epochs

    for epoch in range(num_epochs):
        print('Epoch number:', epoch)

        train_epoch_loss = 0
        test_epoch_loss  = 0

        # For each epoch, do training on the trainset
        # Then evaluate the accuracy on the validation set
        for phase in ['train', 'test']:# iterate on train then test
            print(phase)

            if phase == 'train':
                model.train()
            else:
                model.eval()

            total_loss = 0
            total_examples = 0
            total_correct = 0

            # Loop over training / validation data
            for iteration, (X, y) in enumerate(data_loaders[phase]):
                # Forward
                y = y.long()
                X = X.cuda()
                y = y.cuda()
                scores = model(X)




                # Backward
                loss = torch.nn.functional.cross_entropy(scores, y)


                # Update the network only on training data
                if phase == 'train':

                    train_epoch_loss += loss.item()
                    train_step_losses.append(loss.item())
                    # Backpropagate the loss
                    loss.backward()
                    # Then update the network parameters
                    with torch.no_grad():
                        optimizer.step()
                        optimizer.zero_grad()

                else:
                    test_epoch_loss += loss.item()
                    test_step_losses.append(loss.item())
                # Accuracy
                preds = torch.argmax(scores, dim=1)
                num_correct = torch.sum(preds == y)        

                # Print
                total_loss += loss.item()
                total_correct += num_correct.item()
                total_examples += len(X)
                if iteration % print_every == 0:
                    print(total_loss / total_examples, total_correct / total_examples)

            if phase == 'train':
                train_epoch_losses.append(train_epoch_loss/len(data_loaders['train']))

            else:
                test_epoch_losses.append(test_epoch_loss/len(data_loaders['test']))

In [66]:
train_step_losses  = []
train_epoch_losses = []

test_step_losses  = []
test_epoch_losses = []

print_every = 10
num_epochs = 3

for epoch in range(num_epochs):
    print('Epoch number:', epoch)
    
    train_epoch_loss = 0
    test_epoch_loss  = 0

    # For each epoch, do training on the trainset
    # Then evaluate the accuracy on the validation set
    for phase in ['train', 'test']:# iterate on train then test
        print(phase)

        if phase == 'train':
            model.train()
        else:
            model.eval()

        total_loss = 0
        total_examples = 0
        total_correct = 0

        # Loop over training / validation data
        for iteration, (X, y) in enumerate(data_loaders[phase]):
            # Forward
            y = y.long()
            X = X.cuda()
            y = y.cuda()
            scores = model(X)
            
            
            

            # Backward
            loss = torch.nn.functional.cross_entropy(scores, y)
            
            
            # Update the network only on training data
            if phase == 'train':
                
                train_epoch_loss += loss.item()
                train_step_losses.append(loss.item())
                # Backpropagate the loss
                loss.backward()
                # Then update the network parameters
                with torch.no_grad():
                    optimizer.step()
                    optimizer.zero_grad()
            
            else:
                test_epoch_loss += loss.item()
                test_step_losses.append(loss.item())
            # Accuracy
            preds = torch.argmax(scores, dim=1)
            num_correct = torch.sum(preds == y)        

            # Print
            total_loss += loss.item()
            total_correct += num_correct.item()
            total_examples += len(X)
            if iteration % print_every == 0:
                print(total_loss / total_examples, total_correct / total_examples)
        
        if phase == 'train':
            train_epoch_losses.append(train_epoch_loss/len(data_loaders['train']))
            
        else:
            test_epoch_losses.append(test_epoch_loss/len(data_loaders['test']))

In [57]:
#version one : two epochs, freezed backbone
print("                  step_losses                             epoch_losses             ")
fig, axes = plt.subplots(2, 2, figsize=(10, 5))
axes[0][0].plot(train_step_losses)
axes[0][1].plot(train_epoch_losses)
axes[1][0].plot(test_step_losses)
axes[1][1].plot(test_epoch_losses)

In [60]:
#version two : added - one epochs, freezed backbone
print("                  step_losses                             epoch_losses             ")
fig, axes = plt.subplots(2, 2, figsize=(10, 5))
axes[0][0].plot(train_step_losses)
axes[0][1].plot(train_epoch_losses)
axes[1][0].plot(test_step_losses)
axes[1][1].plot(test_epoch_losses)

In [67]:
#version three : Added three epochs, unfreezed (backbone + head)
print("                  step_losses                             epoch_losses             ")
fig, axes = plt.subplots(2, 2, figsize=(10, 5))
axes[0][0].plot(train_step_losses)
axes[0][1].plot(train_epoch_losses)
axes[1][0].plot(test_step_losses)
axes[1][1].plot(test_epoch_losses)

In [72]:
optimization(5)

In [73]:
#version four : Added five epochs(total eight), unfreezed (backbone + head)
print("                  step_losses                             epoch_losses             ")
fig, axes = plt.subplots(2, 2, figsize=(10, 5))
axes[0][0].plot(train_step_losses)
axes[0][1].plot(train_epoch_losses)
axes[1][0].plot(test_step_losses)
axes[1][1].plot(test_epoch_losses)

In [76]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [77]:
optimization(2)

In [78]:
#version five : Added two epochs(total 10), unfreezed (backbone + head), optimizer = SGD, lr = 0.1, mom = 0.9
print("                  step_losses                             epoch_losses             ")
fig, axes = plt.subplots(2, 2, figsize=(10, 5))
axes[0][0].plot(train_step_losses)
axes[0][1].plot(train_epoch_losses)
axes[1][0].plot(test_step_losses)
axes[1][1].plot(test_epoch_losses)

In [83]:
for param in model.parameters():
    param.requires_grad = False
    
    
for param in model.avgpool.parameters():
    param.requires_grad = True
    
for param in model.classifier.parameters():
    param.requires_grad = True

    

In [84]:
for name, param in model.named_parameters():
    print('Name: ', name,  'Requires_Grad: ', param.requires_grad)

In [85]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
optimization(5)

In [88]:
#version six : freez the body again - 5pochs
print("                  step_losses                             epoch_losses             ")
fig, axes = plt.subplots(2, 2, figsize=(10, 5))
axes[0][0].plot(train_step_losses)
axes[0][1].plot(train_epoch_losses)
axes[1][0].plot(test_step_losses)
axes[1][1].plot(test_epoch_losses)

In [92]:
model = torchvision.models.vgg16(pretrained=True)

In [ ]:
model

In [95]:
model = model.cuda()
optimization(10)

In [97]:
#version six : weights of vgg16, freez the body, 10pochs
print("                  step_losses                             epoch_losses             ")
fig, axes = plt.subplots(2, 2, figsize=(10, 5))
axes[0][0].plot(train_step_losses)
axes[1][0].plot(test_step_losses)

In [98]:
model_path = "./Vgg16_body_freezed_10_epochs.pth"
torch.save(model.state_dict(), model_path)

In [99]:
# Download file
from IPython.display import FileLink 
FileLink(r'./Vgg16_body_freezed_10_epochs.pth')